<h1 align="center">Treating meta information</h1>

https://www.opswat.com/blog/opswat-deep-cdr-now-supports-dicom-file-format

# 실습 1. 메타데이터 읽기

In [3]:
# 0. install pydicom
!pip install pydicom --upgrade

# 1. load pydicom library
import pydicom

# 2. check pydicom library version
print(pydicom.__version__)

# 2. load sample data provided by pydicom library 
from pydicom.data import get_testdata_file
filename = get_testdata_file('MR_small.dcm')

# 3. read a DICOM file
ds = pydicom.dcmread(filename)

Defaulting to user installation because normal site-packages is not writeable
2.3.0


In [4]:
# 4. show meta-data
print(ds)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 190
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.5962.1.1.4.1.1.20040826185059.5457
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.5962.2
(0002, 0013) Implementation Version Name         SH: 'DCTOOL100'
(0002, 0016) Source Application Entity Title     AE: 'CLUNIE1'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['DERIVED', 'SECONDARY', 'OTHER']
(0008, 0012) Instance Creation Date              DA: '20040826'
(0008, 0013) Instance Creation Time              TM: '185434'
(0008, 0014) Instance Creator UID                UI: 1.3.6.1.4.1.5962.3
(0008, 0016) SOP Class UID                  

In [5]:
print("Study Date:", ds.StudyDate)
print("Patient's Name:", ds.PatientName)
print("Patient's Sex:", ds.PatientSex)

Study Date: 20040826
Patient's Name: CompressedSamples^MR1
Patient's Sex: F


# 실습 2. 메타데이터 수정 및 저장

In [ ]:
# 1. Read a DICOM file (메타정보 삭제 여부를 확인하기 위해 새로 불러옴)
ds_anonymized = pydicom.dcmread(filename)

In [ ]:
# 2. Delete desired meta-data
ds_anonymized.PatientName = ''
ds_anonymized.PatientSex = ''

In [ ]:
# 3. Checking for deleted metadata
print("'Before' anonymized")
print("Study Date:", ds.StudyDate)
print("Patient's Name:", ds.PatientName)
print("Patient's Sex:", ds.PatientSex)
print("\n")

print("'After' anonymized")
print("Study Date:", ds_anonymized.StudyDate)
print("Patient's Name:", ds_anonymized.PatientName)
print("Patient's Sex:", ds_anonymized.PatientSex)

In [ ]:
# 4. save revised DICOM file
output_filename = 'anonymized.dcm'
ds_anonymized.save_as(output_filename)

In [ ]:
# 5. Check if meta-data is deleted from modified/saved DICOM
ds_anonymized = pydicom.dcmread('anonymized.dcm')

print("'After' anonymized")
print("Study Date:", ds_anonymized.StudyDate)
print("Patient's Name:", ds_anonymized.PatientName)
print("Patient's Sex:", ds_anonymized.PatientSex)

# 실습 3. SimpleITK로 메타정보 읽기 (비추천)

In [ ]:
# 1. load SimpleITK library
import SimpleITK as sitk

# 2. read DICOM 
filename = get_testdata_file('MR_small.dcm')

reader = sitk.ImageFileReader()
reader.SetFileName(filename)
reader.LoadPrivateTagsOn()

reader.ReadImageInformation()

In [ ]:
# 3. show all meta-data
for k in reader.GetMetaDataKeys():
    v = reader.GetMetaData(k)
    print(f'({k}) = = "{v}"')

In [ ]:
reader.GetMetaData('0028|1051')

# 실습 4. 여러 파일중 CT/MRI 구분하여 폴더에 복사하기

In [32]:
# 1. import libraries
import os # for creating directory
import shutil # for copy, move, delete files/directories

In [39]:
# 2. create directories
os.makedirs(name='MR',mode=0o777, exist_ok=True) # directory name MR is created with 777 permission
os.makedirs(name='CT',mode=0o777, exist_ok=True) # directory name CT is created with 777 permission

In [45]:
# 3. get target files
filename1 = get_testdata_file('MR_small.dcm')
filename2 = get_testdata_file('CT_small.dcm')

files = [filename1,filename2]
files

['/usr/local/lib/python3.9/dist-packages/pydicom/data/test_files/MR_small.dcm',
 '/usr/local/lib/python3.9/dist-packages/pydicom/data/test_files/CT_small.dcm']

In [46]:
# 4. check file meta-data
for file in files:
    ds = pydicom.dcmread(file)
    modality = ds.Modality
    print(f'file:{file}, Modality:{modality}')

file:/usr/local/lib/python3.9/dist-packages/pydicom/data/test_files/MR_small.dcm, Modality:MR
file:/usr/local/lib/python3.9/dist-packages/pydicom/data/test_files/CT_small.dcm, Modality:CT


In [48]:
# 5. copy file to target directory
for file in files:
    ds = pydicom.dcmread(file) # read DICOM
    modality = ds.Modality # get modality data from meta
    if modality == 'MR':
        src = file                      # source file path for copying in case of MR
        dst = 'MR/'+file.split('/')[-1] # target file path for copying in case of MR
    elif modality == 'CT':
        src = file                      # target file path for copying in case of CT
        dst = 'CT/'+file.split('/')[-1] # target file path for copying in case of CT
    shutil.copy(src, dst)    

In [50]:
# 6. Check CT directory
!ls CT

CT_small.dcm


In [51]:
# 7. Check CT directory
!ls MR

MR_small.dcm
